<a href="https://colab.research.google.com/github/anurag0308/HRanalytics/blob/master/HR_Analytics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf 
tf.test.gpu_device_name() 

'/device:GPU:0'

In [2]:
from google.colab import drive 
drive.mount('/mntDrive') 


Mounted at /mntDrive


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [0]:
import pandas as pd
train = pd.read_csv("/content/drive/My Drive/train_LZdllcl.csv")
test = pd.read_csv("/content/drive/My Drive/test_2umaH9m.csv")

In [0]:
#test.head()

In [6]:
test.isnull().sum()

employee_id                0
department                 0
region                     0
education               1034
gender                     0
recruitment_channel        0
no_of_trainings            0
age                        0
previous_year_rating    1812
length_of_service          0
KPIs_met >80%              0
awards_won?                0
avg_training_score         0
dtype: int64

In [7]:
train.isnull().sum()

employee_id                0
department                 0
region                     0
education               2409
gender                     0
recruitment_channel        0
no_of_trainings            0
age                        0
previous_year_rating    4124
length_of_service          0
KPIs_met >80%              0
awards_won?                0
avg_training_score         0
is_promoted                0
dtype: int64

In [8]:
mean1 = train["previous_year_rating"].mean()
train["previous_year_rating"].fillna(mean1,inplace=True)
train["previous_year_rating"].isnull().sum()

0

In [9]:
mean2 = test["previous_year_rating"].mean()
test["previous_year_rating"].fillna(mean2,inplace=True)
test["previous_year_rating"].isnull().sum()

0

In [0]:
from sklearn.preprocessing import Imputer
from statistics import mode

In [11]:
mode(train['previous_year_rating'])


3.0

In [12]:
mode(train['education'])

"Bachelor's"

In [0]:
train['education'] = train['education'].fillna("Bachelor's")
test['education'] = train['education'].fillna("Bachelor's")



In [0]:
train['previous_year_rating'] = train['previous_year_rating'].fillna(3.0)
test['previous_year_rating'] = test['previous_year_rating'].fillna(3.0)

In [15]:
#train['tot_score'] = train['avg_training_score'] * train['no_of_trainings']
#test['tot_score'] = test['avg_training_score'] * test['no_of_trainings']
train.columns

Index(['employee_id', 'department', 'region', 'education', 'gender',
       'recruitment_channel', 'no_of_trainings', 'age', 'previous_year_rating',
       'length_of_service', 'KPIs_met >80%', 'awards_won?',
       'avg_training_score', 'is_promoted'],
      dtype='object')

In [0]:
from sklearn.preprocessing import LabelEncoder
lb_make = LabelEncoder()
train['department'] = lb_make.fit_transform(train['department'])
train['region'] = lb_make.fit_transform(train['region'])
train['gender'] = lb_make.fit_transform(train['gender'])
train['recruitment_channel'] = lb_make.fit_transform(train['recruitment_channel'])
train['education'] = lb_make.fit_transform(list(train['education']))
test['department'] = lb_make.fit_transform(test['department'])
test['region'] = lb_make.fit_transform(test['region'])
test['gender'] = lb_make.fit_transform(test['gender'])
test['recruitment_channel'] = lb_make.fit_transform(test['recruitment_channel'])


In [17]:
test['gender'].head()

0    1
1    0
2    1
3    0
4    1
Name: gender, dtype: int64

In [18]:
train['education'] = lb_make.fit_transform(train['education'])
train['education'].head()

0    2
1    0
2    0
3    0
4    0
Name: education, dtype: int64

In [0]:
test['education'] = lb_make.fit_transform(list(test['education']))

In [20]:
test['education'].head()

0    2
1    0
2    0
3    0
4    0
Name: education, dtype: int64

In [21]:
import pandas_profiling as pp
profile = pp.ProfileReport(train)
profile

/usr/local/lib/python3.6/dist-packages/pandas_profiling/describe.py:392: FutureWarning: The join_axes-keyword is deprecated. Use .reindex or .reindex_like on the result to achieve the same functionality.
  variable_stats = pd.concat(ldesc, join_axes=pd.Index([names]), axis=1)


Number of variables,14
Number of observations,54808
Total Missing (%),0.0%
Total size in memory,5.9 MiB
Average record size in memory,112.0 B
Numeric,10
Categorical,0
Boolean,4
Date,0
Text (Unique),0
Rejected,0


In [0]:
X = train.iloc[:,1:13]
y = train.iloc[:,13]

In [0]:
from sklearn.model_selection import train_test_split

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [25]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=500, max_depth=10, random_state=0)
clf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=10, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=500,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [0]:
y_pred = clf.predict(X_test)

In [0]:
from sklearn.metrics import f1_score

In [28]:
f1_score(y_test, y_pred, average='macro')

0.6196915728651955

In [29]:
test_predictions = clf.predict(test.iloc[:,1:13])
test_predictions



array([0, 0, 0, ..., 0, 0, 0])

In [0]:
df=pd.DataFrame(test.iloc[:,0])
df.insert(1,"is_promoted",test_predictions)
df.to_csv("outputRF.csv",index=False)

In [0]:
import xgboost as xgb

In [32]:
xgb_model = xgb.XGBClassifier(n_estimators=500, max_depth=5, objective="binary:logistic", random_state=42)
xgb_model.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=5,
              min_child_weight=1, missing=None, n_estimators=500, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=42,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [0]:
y_pred_xgb = xgb_model.predict(X_test)

In [34]:
f1_score(y_test, y_pred_xgb, average='macro')

0.7417784426897442

In [35]:
test_predictions_xgb = xgb_model.predict(test.iloc[:,1:13])
test_predictions_xgb

array([0, 0, 0, ..., 0, 0, 1])

In [0]:
df_xgb=pd.DataFrame(test.iloc[:,0])
df_xgb.insert(1,"is_promoted",test_predictions_xgb)
df_xgb.to_csv("outputXGB1.csv",index=False)

In [0]:
from lightgbm import LGBMClassifier

In [38]:
lgb = LGBMClassifier(objective='binary',n_estimators=1000,max_depth=10)
lgb.fit(X_train, y_train)

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=10,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=1000, n_jobs=-1, num_leaves=31, objective='binary',
               random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [0]:
y_pred_lgb = lgb.predict(X_test)

In [40]:
f1_score(y_test, y_pred_lgb, average='macro')

0.7404461188411411